In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt   
import seaborn as sns

In [4]:
df = pd.read_parquet("Data to use/full_esa_data.parquet")

print(df.shape)
print(df.columns)

(81941, 21)
Index(['cosparId', 'vimpelId', 'satno', 'name', 'objectClass', 'mass', 'shape',
       'width', 'height', 'depth', 'diameter', 'span', 'xSectMax', 'xSectMin',
       'xSectAvg', 'firstEpoch', 'mission', 'predDecayDate', 'active',
       'cataloguedFragments', 'onOrbitCataloguedFragments'],
      dtype='object')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81941 entries, 0 to 81940
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cosparId                    67666 non-null  object 
 1   vimpelId                    14933 non-null  float64
 2   satno                       63052 non-null  float64
 3   name                        67390 non-null  object 
 4   objectClass                 81941 non-null  object 
 5   mass                        34050 non-null  float64
 6   shape                       33235 non-null  object 
 7   width                       18430 non-null  float64
 8   height                      32680 non-null  float64
 9   depth                       18403 non-null  float64
 10  diameter                    14809 non-null  float64
 11  span                        32380 non-null  float64
 12  xSectMax                    32980 non-null  float64
 13  xSectMin                    329

In [8]:
df.describe()

,vimpelId,satno,mass,width,height,depth,diameter,span,xSectMax,xSectMin,xSectAvg,cataloguedFragments,onOrbitCataloguedFragments
count,1.493300e+04,63052.000000,34050.000000,18430.000000,32680.000000,18403.000000,14809.00000,32380.000000,32980.000000,32980.000000,32920.000000,81941.000000,81941.000000
mean,9.564043e+04,31543.718455,1807.384763,2.231862,3.278075,1.888098,2.30913,9.730365,28.481166,4.637487,17.281073,0.354926,0.175224
std,4.533195e+04,18220.717310,7269.503234,2.070336,5.173099,2.977385,1.43835,152.026790,101.742724,37.008288,77.054413,17.771147,11.462052
min,9.600000e+03,1.000000,0.001500,0.000000,0.001000,0.012000,0.00100,0.020000,0.000130,0.000000,0.000102,0.000000,0.000000
25%,6.921100e+04,15766.750000,130.820000,0.400000,0.300000,0.500000,1.40000,1.500000,1.375159,0.231100,1.100000,0.000000,0.000000
50%,7.910600e+04,31534.500000,500.000000,2.200000,1.200000,1.500000,2.40000,6.500000,16.219661,0.810000,11.087467,0.000000,0.000000
75%,1.412180e+05,47302.250000,1600.000000,3.700000,5.500000,2.700000,3.00000,8.990000,25.626695,4.523893,17.070629,0.000000,0.000000
max,1.346400e+06,63155.000000,450000.000000,72.800000,60.000000,108.500000,41.14000,19200.000000,8319.848402,1521.000000,5762.400000,3536.000000,2562.000000


**Lot of null values so need to decide what to do with them.  We'll start be removing columns that we won't be using**

In [34]:
columns_to_drop = ['vimpelId', 'satno', 'predDecayDate', 'active', 'mission']

In [ ]:
df.drop(columns = columns_to_drop, inplace = True)

df.isna().sum()

C:\Users\tkbar\AppData\Local\Temp\ipykernel_11068\2164702815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = 'mission', inplace = True)


cosparId                        458
name                            734
objectClass                       0
mass                          34074
shape                         34889
width                         49694
height                        35444
depth                         49721
diameter                      53315
span                          35744
xSectMax                      35144
xSectMin                      35144
xSectAvg                      35204
firstEpoch                     4073
cataloguedFragments               0
onOrbitCataloguedFragments        0
dtype: int64

**If all rows are NA values or 0, drop them and reassess**

In [31]:
# Lot of rows are all NA or 0, but have positive values for first Epoch, so removing those

exclude_col = ['firstEpoch', 'objectClass']

# Drop rows where all values (except the exclude_col) are NA or 0
df = df[~((df.drop(columns=exclude_col).isna()) | (df.drop(columns=exclude_col) == 0)).all(axis=1)]

df.isna().sum()

cosparId                        458
name                            734
objectClass                       0
mass                          34074
shape                         34889
width                         49694
height                        35444
depth                         49721
diameter                      53315
span                          35744
xSectMax                      35144
xSectMin                      35144
xSectAvg                      35204
firstEpoch                     4073
mission                       45409
cataloguedFragments               0
onOrbitCataloguedFragments        0
dtype: int64

**Name and object class are still necessary, so if both are null then remove, otherwise keep and deal with the other missing data as needed**

In [39]:
df_cleaned = df[~df[['cosparId', 'name']].isna().all(axis = 1)]

df_cleaned.isna().sum()

cosparId                        458
name                            734
objectClass                       0
mass                          34074
shape                         34889
width                         49694
height                        35444
depth                         49721
diameter                      53315
span                          35744
xSectMax                      35144
xSectMin                      35144
xSectAvg                      35204
firstEpoch                     4073
cataloguedFragments               0
onOrbitCataloguedFragments        0
dtype: int64

In [42]:
df_cleaned.to_csv("Cleaned_Data/cleaned_esa.csv", index = False)